In [28]:
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
Blinks = np.load('/content/drive/MyDrive/DLA/npy/Blinks_F3.npy')
Labels = np.load('/content/drive/MyDrive/DLA/npy/Labels_F3.npy')
BlinksTest = np.load('/content/drive/MyDrive/DLA/npy/BlinksTest_F3.npy')
LabelsTest = np.load('/content/drive/MyDrive/DLA/npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [48]:
#BinarizationLabels
LabelsTest[LabelsTest==10] = 1
Labels[Labels==10] = 1

In [32]:
def create_model(model_name):
    if model_name == 'LSTMreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'LSTM':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Masking(mask_value=0.0)(input)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.5)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'stackedreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      rnn_cells = [tf.keras.layers.LSTMCell(32) for _ in range(4)]
      stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
      x = tf.keras.layers.RNN(stacked_lstm)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    else:
      print("Chiama le cose con il loro nome")
    return model

In [49]:
model_name = "stackedreza"
model = create_model(model_name)

In [50]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 30, 4)]           0         
_________________________________________________________________
dense_25 (Dense)             (None, 30, 64)            320       
_________________________________________________________________
masking_5 (Masking)          (None, 30, 64)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_26 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_10 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 16)                272 

In [51]:
lr = 0.000053
bs = 64
ep = 100
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
)

In [ ]:
model.fit(x= Blinks, y = Labels, validation_data=(BlinksTest, LabelsTest), epochs=ep, batch_size=bs)

Epoch 1/100
121/121 [==============================] - 10s 54ms/step - loss: 1.5181 - accuracy: 0.4545 - val_loss: 1.0196 - val_accuracy: 0.2999
Epoch 2/100
121/121 [==============================] - 6s 49ms/step - loss: 1.0495 - accuracy: 0.4643 - val_loss: 0.7216 - val_accuracy: 0.6459
Epoch 3/100
121/121 [==============================] - 6s 49ms/step - loss: 0.8336 - accuracy: 0.5048 - val_loss: 0.5836 - val_accuracy: 0.6877
Epoch 4/100
121/121 [==============================] - 6s 49ms/step - loss: 0.7442 - accuracy: 0.5511 - val_loss: 0.5230 - val_accuracy: 0.7117
Epoch 5/100
121/121 [==============================] - 6s 49ms/step - loss: 0.7023 - accuracy: 0.5570 - val_loss: 0.5007 - val_accuracy: 0.7312
Epoch 6/100
121/121 [==============================] - 6s 49ms/step - loss: 0.6982 - accuracy: 0.5619 - val_loss: 0.4896 - val_accuracy: 0.7494
Epoch 7/100
121/121 [==============================] - 6s 49ms/step - loss: 0.6910 - accuracy: 0.5640 - val_loss: 0.4803 - val_accuracy

In [38]:
model.fit(x= Blinks, y = Labels, validation_data=(BlinksTest, LabelsTest), epochs=100, batch_size=128)

Epoch 1/100
61/61 [==============================] - 5s 78ms/step - loss: 0.6880 - accuracy: 0.5636 - val_loss: 0.4776 - val_accuracy: 0.7730
Epoch 2/100
61/61 [==============================] - 4s 67ms/step - loss: 0.6883 - accuracy: 0.5671 - val_loss: 0.4763 - val_accuracy: 0.7757
Epoch 3/100
61/61 [==============================] - 4s 68ms/step - loss: 0.6840 - accuracy: 0.5707 - val_loss: 0.4744 - val_accuracy: 0.7761
Epoch 4/100
61/61 [==============================] - 4s 68ms/step - loss: 0.6862 - accuracy: 0.5657 - val_loss: 0.4733 - val_accuracy: 0.7770
Epoch 5/100
61/61 [==============================] - 4s 68ms/step - loss: 0.6840 - accuracy: 0.5698 - val_loss: 0.4719 - val_accuracy: 0.7770
Epoch 6/100
61/61 [==============================] - 4s 69ms/step - loss: 0.6884 - accuracy: 0.5619 - val_loss: 0.4707 - val_accuracy: 0.7770
Epoch 7/100
61/61 [==============================] - 4s 67ms/step - loss: 0.6859 - accuracy: 0.5729 - val_loss: 0.4692 - val_accuracy: 0.7783
Epoch 

In [ ]:
tf.compat.v1.disable_eager_execution()
training = tf.compat.v1.placeholder(tf.bool,name='phase_train')